### DB 연결

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
import time
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
from impyute.imputation.cs import mice
#from sqlalchemy import create_engine

In [2]:
#pip freeze > requirements.txt

In [5]:
import psycopg2
from sshtunnel import SSHTunnelForwarder


try:
    tunnel= SSHTunnelForwarder(
         ('210.91.223.248', 35430),
         #ssh_private_key="</path/to/private/ssh/key>",
         ### in my case, I used a password instead of a private key
         ssh_username="medicalai",
         ssh_password="medicalai731!", 
         remote_bind_address=('localhost', 35432))
         
    tunnel.start()
    print("tunnel connected")

    params = {
             'database': 'mimiciv',
             'user': 'mai_admin',
             'password': 'admin1q2w!@',
             'host': 'localhost',
             'port': tunnel.local_bind_port
             }

    conn = psycopg2.connect(**params)
    conn.autocommit = True
    curs = conn.cursor()
    print("database connected")

except Exception as e:
    print("Connection Failed due to {e}")

# Remember to close the SSH tunnel
#tunnel.close()


tunnel connected
database connected


### 테이블확인

In [20]:
import psycopg2
import os

# 사용 가능한 모든 스키마 조회
curs.execute("SELECT DISTINCT schemaname FROM pg_tables WHERE schemaname NOT IN ('pg_catalog', 'information_schema');")
schemanames = curs.fetchall()
schemanames


[('mimiciv_hosp',), ('mimiciv_derived',), ('mimiciv_icu',), ('mimiciv_note',)]

In [22]:
#어떤 스키마에 어떤 테이블이 있는지 확인
curs.execute("select * from PG_TABLEs;")
rows = curs.fetchall( ) #결과가지고옴

for row in rows:
    print(row)

# 파일에 결과 저장
with open("db_tables.txt", "w") as file:
    for row in rows:
        # 각 row의 값들을 문자열로 변환하고 콤마로 구분하여 한 줄로 만듬
        line = ", ".join(map(str, row))
        file.write(line + "\n")  # 파일에 쓰고, 다음 줄로 넘어감


('mimiciv_hosp', 'diagnoses_icd', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'labevents', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'd_icd_diagnoses', 'mai_admin', None, True, False, False, False)
('mimiciv_hosp', 'd_icd_procedures', 'mai_admin', None, True, False, False, False)
('mimiciv_hosp', 'emar', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'hcpcsevents', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'd_labitems', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'pharmacy', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'omr', 'mai_admin', None, False, False, False, False)
('mimiciv_hosp', 'poe_detail', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'poe', 'mai_admin', None, True, False, True, False)
('pg_catalog', 'pg_statistic', 'postgres', None, True, False, False, False)
('pg_catalog', 'pg_type', 'postgres', None, True, False, False, False)
('mimiciv_hosp', 'pro

In [ ]:
# 어떤 변수들부터 뽑아볼까?
# ventilator mode
# Peak pressure(cmH2O)
# Plateau pressure(cmH2O)
# Mean pressure(cmH2O)
# PEEP ((cmH2O))
# Tidal volume (mL)
# FiO2 (%)
# Phigh(cmH2O)
# Plow(cmH2O)
# Thigh(sec)
# Tlow(sec)

In [ ]:
#curs.execute("select * from mimiciv_derived.gcs;")
#columns_name = [desc[0] for desc in curs.description]
#gcs = pd.DataFrame(curs.fetchall(), columns=columns_name) 
#gcs

,subject_id,stay_id,charttime,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable
0,12466550,30000153,2174-09-29 12:45:00,15.0,5.0,0.0,3.0,1
1,12466550,30000153,2174-09-29 16:26:00,15.0,6.0,0.0,4.0,1
2,12466550,30000153,2174-09-29 17:37:00,15.0,6.0,0.0,4.0,1
3,12466550,30000153,2174-09-29 18:00:00,9.0,5.0,1.0,3.0,0
4,12466550,30000153,2174-09-29 19:00:00,9.0,5.0,1.0,3.0,0
...,...,...,...,...,...,...,...,...
1637758,17840864,39999810,2115-12-05 00:00:00,14.0,6.0,5.0,3.0,0
1637759,17840864,39999810,2115-12-05 04:00:00,14.0,6.0,5.0,3.0,0
1637760,17840864,39999810,2115-12-05 08:28:00,15.0,6.0,5.0,4.0,0
1637761,17840864,39999810,2115-12-05 12:00:00,15.0,6.0,5.0,4.0,0


In [ ]:
#gcs.to_csv('../outputs/gcs.csv', index=False, encoding='utf-8-sig')

In [ ]:
gcs = pd.read_csv('../outputs/gcs.csv', parse_dates=['charttime'])

In [ ]:
gcs

,subject_id,stay_id,charttime,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable
0,12466550,30000153,2174-09-29 12:45:00,15.0,5.0,0.0,3.0,1
1,12466550,30000153,2174-09-29 16:26:00,15.0,6.0,0.0,4.0,1
2,12466550,30000153,2174-09-29 17:37:00,15.0,6.0,0.0,4.0,1
3,12466550,30000153,2174-09-29 18:00:00,9.0,5.0,1.0,3.0,0
4,12466550,30000153,2174-09-29 19:00:00,9.0,5.0,1.0,3.0,0
...,...,...,...,...,...,...,...,...
1637758,17840864,39999810,2115-12-05 00:00:00,14.0,6.0,5.0,3.0,0
1637759,17840864,39999810,2115-12-05 04:00:00,14.0,6.0,5.0,3.0,0
1637760,17840864,39999810,2115-12-05 08:28:00,15.0,6.0,5.0,4.0,0
1637761,17840864,39999810,2115-12-05 12:00:00,15.0,6.0,5.0,4.0,0


In [ ]:
# curs.execute("select * from mimiciv_hosp.d_icd_diagnoses;")
# columns_name = [desc[0] for desc in curs.description]
# d_icd_diagnoses = pd.DataFrame(curs.fetchall(), columns=columns_name) 
# d_icd_diagnoses

In [ ]:
# curs.execute("select * from mimiciv_hosp.diagnoses_icd;")
# columns_name = [desc[0] for desc in curs.description]
# diagnoses_icd = pd.DataFrame(curs.fetchall(), columns=columns_name) 
# diagnoses_icd

In [ ]:
# curs.execute("select * from mimiciv_hosp.d_labitems;")
# columns_name = [desc[0] for desc in curs.description]
# d_labitems = pd.DataFrame(curs.fetchall(), columns=columns_name) 
# d_labitems

In [ ]:
# curs.execute("select * from mimiciv_hosp.d_icd_procedures;")
# columns_name = [desc[0] for desc in curs.description]
# d_icd_procedures = pd.DataFrame(curs.fetchall(), columns=columns_name) 
# d_icd_procedures

,icd_code,icd_version,long_title
0,0001,9,Therapeutic ultrasound of vessels of head and ...
1,0002,9,Therapeutic ultrasound of heart
2,0003,9,Therapeutic ultrasound of peripheral vascular ...
3,0009,9,Other therapeutic ultrasound
4,001,10,"Central Nervous System and Cranial Nerves, Bypass"
...,...,...,...
85252,XW0DXV5,10,Introduction of Gilteritinib Antineoplastic in...
85253,XXE,10,"New Technology, Physiological Systems, Measure..."
85254,XXE5XM5,10,"Measurement of Infection, Whole Blood Nucleic ..."
85255,XY0,10,"New Technology, Extracorporeal, Introduction"


In [ ]:
# curs.execute("select * from mimiciv_hosp.procedures_icd;")
# columns_name = [desc[0] for desc in curs.description]
# procedures_icd = pd.DataFrame(curs.fetchall(), columns=columns_name) 
# procedures_icd

,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version
0,10000032,22595853,1,2180-05-07,5491,9
1,10000032,22841357,1,2180-06-27,5491,9
2,10000032,25742920,1,2180-08-06,5491,9
3,10000068,25022803,1,2160-03-03,8938,9
4,10000117,27988844,1,2183-09-19,0QS734Z,10
...,...,...,...,...,...,...
669181,19999840,21033226,5,2164-09-16,0331,9
669182,19999840,26071774,1,2164-07-25,8891,9
669183,19999840,26071774,2,2164-07-25,8841,9
669184,19999987,23865745,1,2145-11-07,8841,9


In [ ]:
curs.execute("select * from mimiciv_icu.d_items;")
columns_name = [desc[0] for desc in curs.description]
d_items = pd.DataFrame(curs.fetchall(), columns=columns_name) 
d_items

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220001,Problem List,Problem List,chartevents,General,None,Text,NaN,NaN
1,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,None,Date and time,NaN,NaN
2,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
3,220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN
4,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4009,230077,Documented on DC Summary (PICC),Documented on DC Summary (PICC),chartevents,PICC Line Insertion,None,Text,NaN,NaN
4010,230078,Documented on DC Summary (Thora),Documented on DC Summary (Thora),chartevents,Thoracentesis,None,Text,NaN,NaN
4011,230083,Replacement Fluid (Pre-Filter),Replacement Fluid (Pre-Filter),chartevents,Dialysis,None,Text,NaN,NaN
4012,230084,Replacement Fluid (Post-Filter),Replacement Fluid (Post-Filter),chartevents,Dialysis,None,Text,NaN,NaN


In [ ]:
d_items[d_items['label'].str.contains('ressure')] #intubation, extubation 은 chartevents 에 있는 것으로 봐야 할 듯 함. // Procedureevents 에도 있음 

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
6,220050,Arterial Blood Pressure systolic,ABPs,chartevents,Routine Vital Signs,mmHg,Numeric,90.0,140.0
7,220051,Arterial Blood Pressure diastolic,ABPd,chartevents,Routine Vital Signs,mmHg,Numeric,60.0,90.0
8,220052,Arterial Blood Pressure mean,ABPm,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
9,220056,Arterial Blood Pressure Alarm - Low,ABP Alarm - Low,chartevents,Alarms,mmHg,Numeric,NaN,NaN
10,220058,Arterial Blood Pressure Alarm - High,ABP Alarm - High,chartevents,Alarms,mmHg,Numeric,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3946,229891,Pressure Ulcer #8- Date Indentified,Pressure Ulcer #8- Date Indentified,datetimeevents,Skin - Impairment,None,Date and time,NaN,NaN
3947,229892,Pressure Ulcer #9- Date Indentified,Pressure Ulcer #9- Date Indentified,datetimeevents,Skin - Impairment,None,Date and time,NaN,NaN
3948,229893,Pressure Ulcer #10- Date Indentified,Pressure Ulcer #10- Date Indentified,datetimeevents,Skin - Impairment,None,Date and time,NaN,NaN
3954,229899,Left Ventricular Pressure Signal - Systolic,Left Ventricular Pressure Signal - Systolic,chartevents,Impella,mmHg,Numeric,NaN,NaN


In [ ]:
d_items[d_items['label'].str.contains('Blood Pressure')] 
#itemid 220050 : ABPs (arterial blood pressure systolic), 220051 : ABPm (arterial blood pressure mean), 220052 : ABPd (arterial blood pressure diastolic) #90~140
#itemid 220179 : NBP (non-invasive blood pressure systolic), 220180 : NBPm (non-invasive blood pressure mean), 220181 : NBPd (non-invasive blood pressure diastolic) #60~90
#-----------------------------------
#itemid 224167 Manual BP systolic left, 227242 Manual BP systolic right
#itemid 224643 Manual BP diastolic left, 227243 Manual BP diastolic right

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
6,220050,Arterial Blood Pressure systolic,ABPs,chartevents,Routine Vital Signs,mmHg,Numeric,90.0,140.0
7,220051,Arterial Blood Pressure diastolic,ABPd,chartevents,Routine Vital Signs,mmHg,Numeric,60.0,90.0
8,220052,Arterial Blood Pressure mean,ABPm,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
9,220056,Arterial Blood Pressure Alarm - Low,ABP Alarm - Low,chartevents,Alarms,mmHg,Numeric,NaN,NaN
10,220058,Arterial Blood Pressure Alarm - High,ABP Alarm - High,chartevents,Alarms,mmHg,Numeric,NaN,NaN
24,220179,Non Invasive Blood Pressure systolic,NBPs,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
25,220180,Non Invasive Blood Pressure diastolic,NBPd,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
26,220181,Non Invasive Blood Pressure mean,NBPm,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
331,223751,Non-Invasive Blood Pressure Alarm - High,NBP Alarm - High,chartevents,Alarms,mmHg,Numeric,NaN,NaN
332,223752,Non-Invasive Blood Pressure Alarm - Low,NBP Alarm - Low,chartevents,Alarms,mmHg,Numeric,NaN,NaN


In [ ]:
#chartevents 에서 220050, 220051, 220052, 220179, 220180, 220181, 224167, 227242, 224643, 227243 를 찾아야 함

curs.execute("select * from mimiciv_icu.chartevents where itemid in (220050, 220051, 220052, 220179, 220180, 220181, 224167, 227242, 224643, 227243);")
columns_name = [desc[0] for desc in curs.description]
bp_all = pd.DataFrame(curs.fetchall(), columns=columns_name) 
bp_all


MemoryError: Unable to allocate 870. MiB for an array with shape (6, 19002628) and data type int64

In [ ]:
bp_all.to_csv('../outputs/bp_all.csv', index=False, encoding='utf-8-sig')

In [ ]:
bp_all = pd.read_csv('../outputs/bp_all.csv', parse_dates=['charttime'])

In [ ]:

# Remember to close the SSH tunnel
tunnel.close()